In [1]:
import random
import requests as r
from requests import exceptions as req_error
from faker import Faker
from bs4 import BeautifulSoup as bs
import random
agent = Faker('zh-CN').user_agent()
def nation_city():
    
    proxies = [{'http':'http://47.107.175.190:8000'},
          {'http':'http://163.204.242.64:9999'},
          {'http':'http://47.112.214.45:8000'},
        {'http':'http://163.204.242.181:9999'},
        {'http':'http://163.204.241.214:9999'},
        {'http':'http://58.62.44.103:8088'}]

    city_json = 'https://www.zhipin.com/wapi/zpCommon/data/city.json'

    pro = random.choice(proxies)

    adv = r.get(city_json,
                headers={'User-Agent':agent},
               proxies=pro).json()['zpData']['cityList']

    # 创建一个市级行政区域与省级行政区的映射，方便后面统计各省的职位数量
    pt = {}
    for ad in adv:
        for a in ad['subLevelModelList']:
            pt[a['name']] = ad['name']
    return pt

In [2]:
def get_cityCode(province='',city=''):
    city_json = 'https://www.zhipin.com/wapi/zpCommon/data/city.json'
    
    proxies = [{'http':'http://47.107.175.190:8000'},
                {'http':'http://163.204.242.64:9999'},
                {'http':'http://47.112.214.45:8000'},
                {'http':'http://163.204.242.181:9999'},
                {'http':'http://163.204.241.214:9999'},
                {'http':'http://58.62.44.103:8088'}]
    pro = random.choice(proxies)
    result = r.get(city_json,
                headers={'User-Agent':agent},
                  proxies=pro).json()
    if (result['message'] == 'Success'):
        cityList = result['zpData']['cityList']
        for p in cityList:
            if p['name'] == province:
                sub = p['subLevelModelList']
                flag = 1
                for c in sub:
                    if city == c['name']:
                        return p['code'],c['code']  
                return p['code']
        else:
            print('找不到对象')
            return None
    
    else:
        print('调用boss直聘的获取地区编码的api有问题！',city)

In [3]:
def get_job(job,city):
    proxies = [{'http':'http://47.107.175.190:8000'},
          {'http':'http://163.204.242.64:9999'},
          {'http':'http://47.112.214.45:8000'},
        {'http':'http://163.204.242.181:9999'},
        {'http':'http://163.204.241.214:9999'},
        {'http':'http://60.12.216.5:8088'},
        {'http':'http://218.16.127.84:8088'}]
    
    job_json = 'https://www.zhipin.com/mobile/jobs.json'
    params = {
        'query':job,
        'page':1,
        'city':city
    }
    job_data = []
    while 1:
        pro = random.choice(proxies)
        response = r.get(job_json,
              params=params,
              headers={'User-Agent':agent},
            proxies=pro).json()
        
        try:
            
            if response['html']:
                params['page'] +=1
                html = bs(response['html'],'html.parser').find_all(class_='item')
                for h in html:
                # item字典一定要写在循环里面且每遍历完一个职位条目就重新清空字典、保存在job列表当中
                # 否则会重复将最后的值，反复追加到job列表，可见变量名只是个字典的映射。
                    item = {}
                    item['salary'] = h.find('span',"salary").string 
                    item['company'] = h.find('div','name').string
                    item['jobName'] = h.find('div','title').find('h4').string
                    msg = h.find('div','msg').find_all('em')
                    for em in range(3):
                        item['city'] = msg[0].string
                        item['experience'] = msg[1].string
                        item['education'] = msg[2].string
                    job_data.append(item)
            elif response['resmsg'] != '成功':
                print('调用boss直聘获取工作列表api有问题！或者是网络、服务器有问题')
                continue
            elif response['html'] == '' and params['page']==1:
                print('用户输入的关键词有误，找不到对象！')
                return None
                break
            else:
                break
        # 返回的json数据结果没有html键，说明ip已被封
        except KeyError:
            print(response)
            break
        # 连接代理服务出现异常时，就把那个代理ip从列表中删除并继续用另一个代理ip
        except (req_error.ConnectTimeout,req_error.ProxyError,req_error.ReadTimeout):
            proxies.remove(pro)
            continue
    return job_data

In [6]:
def generate_jsonFile(job_item=['区块链','大数据','人工智能','物联网','云计算'],
                      job_data = {'salary':[],'jobName':[],'education':[],'experience':[]}):
    import json
    city = nation_city()
    for c in city:
        # 一定要给准确的市级编码，否则只会给出北京的数据
        cityCode = get_cityCode(city[c],c)[1]
        for job in job_item:
            jobData = get_job(job,cityCode)
            for j in jobData:
                # 招聘网站会拿其它城市的数据来充数
                if j['city']== c:
                    job_data['salary'].append(j['salary'])
                    job_data['jobName'].append(j['jobName'])
                    job_data['education'].append(j['education'])
                    job_data['experience'].append(j['experience'])
                    fileName = job+'_'+city[c]+'_'+c+'.json'
                    with open('json/'+fileName,mode='w') as js:
                        js.write(json.dumps(job_data,indent=4))   
                else:
                    break

In [ ]:
# fkey [大数据 人工智能 区块链 物联网 云计算 ]
# kw ['salary' 'education' 'jobname' 'experience']
def countNum(fkey,kw):
    try:
        file = glob.glob('json/'+fkey+'_*_*.json')
        edu_num = {kw:{}}
        for ff in file:
            with open(ff) as f:
                content = json.loads(f.read())
                for j in content[kw]:
                    edu_num[kw][j] = edu_num[kw].get(j,0)+1
        return sorted(edu_num[kw].items(),key= lambda x:x[1],reverse=True)
    except KeyError:
        print("请输入正确的关键词\n fkey [大数据 人工智能 区块链 物联网 云计算 ]\n 请输入正确的关键词\n fkey [大数据 人工智能 区块链 物联网 云计算 ]\n")